# Big Red Soundscape
Notebook Playground 2  
_by Ahmed Sultan AND CARLY HU, for Cornell Data Journal_

In [149]:
import json
import math

import numpy as np
import pandas as pd
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import matplotlib.pyplot as plt
import seaborn as sns

In [150]:
# update access keys here for Spotify API here :) 
client_id = '860ada55cc094dbf906250fe2d4caf48'
client_secret = '75fe4137c4a54541800d19b74dc706fb'

# sp is how we interface with the Spotify API
# read the docs here: https://spotipy.readthedocs.io/en/2.22.1/
sp = spotipy.Spotify(client_credentials_manager=\
                     SpotifyClientCredentials(client_id, client_secret))

In [151]:
# Big Data, Machine Learning...
# https://www.youtube.com/watch?v=bqqCTC9nQDY

# let's aggregate an entire user's public library of songs
playlists = []
LIMIT = 20
selected_user = 'mittfard'

query = sp.user_playlists(selected_user, limit=LIMIT)

n = 0

while len(query['items']) != 0:
    n += LIMIT
    for i in query['items']:
        playlists.append({'title': i['name'],'id': i['id'], 'length': i['tracks']['total']})
    if LIMIT == len(query['items']):
        query = sp.user_playlists(selected_user, limit=LIMIT, offset=n)
    else:
        break
        
# there's definitely a cleaner way to do this that we'll figure out
# im definitely lazy

In [152]:
playlists

[{'title': 'breakdown', 'id': '2POGswOpqoTIgAurmdEcyc', 'length': 45},
 {'title': "qbert's final stand",
  'id': '2cXSVfvLNhsOkf2H9Z1t5c',
  'length': 10},
 {'title': 'My Playlist #58', 'id': '7vOftmTSDEi6CoTVDXkpnx', 'length': 3701},
 {'title': 'quirked up white boy anthems',
  'id': '2A7E3Qf7cDdmvcgTPtSo66',
  'length': 0},
 {'title': 'cdj anthology', 'id': '5yiyV59j0MgWmOw85LkBAj', 'length': 5},
 {'title': 'torontgo', 'id': '4yfGFvrZ6cFi3BRV6zHqwX', 'length': 78},
 {'title': 'reconnaissance', 'id': '41v4e5mHvox6Z9Cgw7RKuq', 'length': 12},
 {'title': 'winter 22-23', 'id': '5arZaaT0vVmbATHhY2ApTQ', 'length': 19},
 {'title': 'sultanshocker', 'id': '6PBLr4yQMFvA1F29w0yjNL', 'length': 20},
 {'title': 'the burning west', 'id': '3eAN10OhUTApqY8ZjNqRYf', 'length': 34},
 {'title': 'tribulations', 'id': '1r1cUo2YmVFD4fr8LmR02u', 'length': 37},
 {'title': 'holographic', 'id': '4Yln2F3tsIcS4SPBWRuC0O', 'length': 50},
 {'title': 'xerox', 'id': '6vsGnUcUIhaHyvnyVpmSSU', 'length': 58},
 {'title': 

In [153]:
# number of public playlists
print(len(playlists))

52


In [154]:
# making sure we can get all songs in a playlist rather than just the first 100
# https://stackoverflow.com/questions/55690063/is-there-a-method-to-retrieve-all-tracks-from-a-playlist-using-spotipy 

def user_playlist_tracks_full(user, playlist_id=None, fields=None):
    """ Get full details of the tracks of a playlist owned by a user.
        https://developer.spotify.com/documentation/web-api/reference/playlists/get-playlists-tracks/

        Parameters:
            - user - the id of the user
            - playlist_id - the id of the playlist
            - fields - which fields to return
    """

    # first run through also retrieves total no of songs in library
    response = sp.user_playlist_tracks(user, playlist_id, fields=None, limit=100)
    results = response["items"]

    # subsequently runs until it hits the user-defined limit or has read all songs in the library
    while len(results) < response["total"]:
        response = sp.user_playlist_tracks(
            user, playlist_id, fields=fields, limit=100, offset=len(results)
        )
        results.extend(response["items"])
    
    return results
        

In [155]:
# now, let's go through each playlist
# this will basically be the same as before

songs = []

for p in playlists:
    
    results = sp.playlist(p['id'])
    
    extended_results = user_playlist_tracks_full(user = selected_user, playlist_id = results['id']) # more than 100
    
    for r in extended_results:
        songs.append(r['track']['id']) 

In [148]:
print(len(songs)) # total songs

7221
